In [23]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [24]:
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [25]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [26]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [27]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [28]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [29]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!',
 'explanation': 'The joke plays on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working dough with your hands to develop gluten and give it a smooth, elastic texture. This is a physically demanding and repetitive task.\n*   **Figurative meaning:** "Kneaded" sounds like "needed." The joke implies the pizza maker was tired of being constantly in demand or feeling like he was being taken advantage of. He was "kneaded" in the sense that his labor was essential and perhaps he felt overworked or unappreciated.\n\nThe humor comes from the unexpected connection between the physical act of kneading dough and the feeling of being needed or pressured in a job. It\'s a pun, relying on the similar sound of the word to create a humorous and relatable scenario.'}

In [30]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke plays on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working dough with your hands to develop gluten and give it a smooth, elastic texture. This is a physically demanding and repetitive task.\n*   **Figurative meaning:** "Kneaded" sounds like "needed." The joke implies the pizza maker was tired of being constantly in demand or feeling like he was being taken advantage of. He was "kneaded" in the sense that his labor was essential and perhaps he felt overworked or unappreciated.\n\nThe humor comes from the unexpected connection between the physical act of kneading dough and the feeling of being needed or pressured in a job. It\'s a pun, relying on the similar sound of the word to create a humorous and relatable scenario.'}, next=(), config

In [31]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke plays on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working dough with your hands to develop gluten and give it a smooth, elastic texture. This is a physically demanding and repetitive task.\n*   **Figurative meaning:** "Kneaded" sounds like "needed." The joke implies the pizza maker was tired of being constantly in demand or feeling like he was being taken advantage of. He was "kneaded" in the sense that his labor was essential and perhaps he felt overworked or unappreciated.\n\nThe humor comes from the unexpected connection between the physical act of kneading dough and the feeling of being needed or pressured in a job. It\'s a pun, relying on the similar sound of the word to create a humorous and relatable scenario.'}, next=(), confi

In [32]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti blush?\n\nBecause he saw the salad dressing!',
 'explanation': 'The joke plays on the double meaning of "dressing."\n\n*   **Literal meaning:** Dressing is a sauce or condiment used to flavor salads.\n*   **Figurative meaning:** "Dressing" can also refer to clothing or lack thereof.\n\nThe joke implies that the spaghetti blushed because he saw the salad "dressing" in the sense of the salad being undressed or naked. The spaghetti is embarrassed by this immodest sight. It\'s a silly and slightly suggestive pun.'}

In [33]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke plays on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working dough with your hands to develop gluten and give it a smooth, elastic texture. This is a physically demanding and repetitive task.\n*   **Figurative meaning:** "Kneaded" sounds like "needed." The joke implies the pizza maker was tired of being constantly in demand or feeling like he was being taken advantage of. He was "kneaded" in the sense that his labor was essential and perhaps he felt overworked or unappreciated.\n\nThe humor comes from the unexpected connection between the physical act of kneading dough and the feeling of being needed or pressured in a job. It\'s a pun, relying on the similar sound of the word to create a humorous and relatable scenario.'}, next=(), config

In [34]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke plays on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working dough with your hands to develop gluten and give it a smooth, elastic texture. This is a physically demanding and repetitive task.\n*   **Figurative meaning:** "Kneaded" sounds like "needed." The joke implies the pizza maker was tired of being constantly in demand or feeling like he was being taken advantage of. He was "kneaded" in the sense that his labor was essential and perhaps he felt overworked or unappreciated.\n\nThe humor comes from the unexpected connection between the physical act of kneading dough and the feeling of being needed or pressured in a job. It\'s a pun, relying on the similar sound of the word to create a humorous and relatable scenario.'}, next=(), confi

### Time Travel

In [37]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f099891-4d24-69ae-8000-f7447b3d698f"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f099891-4d24-69ae-8000-f7447b3d698f'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-24T20:57:28.738039+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099891-4c83-6402-bfff-dad0ef61f045'}}, tasks=(PregelTask(id='ef426e5c-c144-c039-5310-c64bf48d3fb3', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!'}),), interrupts=())

In [38]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f099891-4d24-69ae-8000-f7447b3d698f"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!',
 'explanation': 'The joke plays on the double meaning of the word "dough".\n\n* **Literal Meaning:** In the context of a pizza maker, "dough" refers to the uncooked mixture of flour, water, and other ingredients that is used to make pizza crust. Handling dough is a fundamental part of the job.\n\n* **Figurative Meaning:** "Dough" is also a slang term for money.\n\nThe joke implies that the pizza maker quit because he was tired of receiving money (dough) as payment for his work. The humor comes from the unexpected and absurd twist – while everyone expects to get paid for their job, the pizza maker is portrayed as being fed up with it, using the pizza-related "dough" as the punchline.'}

In [39]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!', 'explanation': 'The joke plays on the double meaning of the word "dough".\n\n* **Literal Meaning:** In the context of a pizza maker, "dough" refers to the uncooked mixture of flour, water, and other ingredients that is used to make pizza crust. Handling dough is a fundamental part of the job.\n\n* **Figurative Meaning:** "Dough" is also a slang term for money.\n\nThe joke implies that the pizza maker quit because he was tired of receiving money (dough) as payment for his work. The humor comes from the unexpected and absurd twist – while everyone expects to get paid for their job, the pizza maker is portrayed as being fed up with it, using the pizza-related "dough" as the punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099893-1aca-6988-8002-989db492b53c'}}, metadata={'source': 'loop', 'step': 2, 'pa

#### Updating State

In [40]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f099891-4d24-69ae-8000-f7447b3d698f", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f099894-9031-6b16-8001-4271b2c71b5c'}}

In [41]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099894-9031-6b16-8001-4271b2c71b5c'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-09-24T20:58:56.299496+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099891-4d24-69ae-8000-f7447b3d698f'}}, tasks=(PregelTask(id='82b2c8da-1f9f-6ec9-ca27-8b3cb13dc7a9', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough!', 'explanation': 'The joke plays on the double meaning of the word "dough".\n\n* **Literal Meaning:** In the context of a pizza maker, "dough" refers to the uncooked mixture of flour, water, and other ingredients that is used to make pizza crust. Han

In [42]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f099894-9031-6b16-8001-4271b2c71b5c"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it said, "I need some space, I feel like you\'re always all over me!"',
 'explanation': 'The joke plays on the common pairing of samosas and chutney as food items, and uses a double meaning of the phrase "I need some space, I feel like you\'re always all over me."\n\nHere\'s the breakdown:\n\n* **Literal Meaning:** Samosas and chutney are often eaten together. The chutney is typically used as a dip or topping for the samosa. Therefore, the chutney is literally "all over" the samosa.\n\n* **Figurative Meaning:** The phrase "I need some space, I feel like you\'re always all over me!" is a common expression used in romantic relationships to indicate that one person feels suffocated or overwhelmed by the other\'s constant presence or attention.\n\nThe humor comes from applying this romantic relationship phrase to the food pairing. The samosa is anthropomorphized, given human emotions, and uses the phrase

In [43]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it said, "I need some space, I feel like you\'re always all over me!"', 'explanation': 'The joke plays on the common pairing of samosas and chutney as food items, and uses a double meaning of the phrase "I need some space, I feel like you\'re always all over me."\n\nHere\'s the breakdown:\n\n* **Literal Meaning:** Samosas and chutney are often eaten together. The chutney is typically used as a dip or topping for the samosa. Therefore, the chutney is literally "all over" the samosa.\n\n* **Figurative Meaning:** The phrase "I need some space, I feel like you\'re always all over me!" is a common expression used in romantic relationships to indicate that one person feels suffocated or overwhelmed by the other\'s constant presence or attention.\n\nThe humor comes from applying this romantic relationship phrase to the food pairing. The samosa is anthropomorphized, given human emotions,

### Fault Tolerance

In [44]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [45]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [46]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(30)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [47]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [48]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [49]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [50]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099899-31c9-6f50-8003-c42cf9e4b9fe'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-24T21:01:00.618120+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099899-31c2-6a17-8002-ab96ca21cd7f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099899-31c2-6a17-8002-ab96ca21cd7f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-24T21:01:00.615119+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099896-bfd2-6819-8001-1e288c36a9bb'}}, tasks=(PregelTask(id='f3b15993-6fa8-9807-3ba2-9c7e9224bbf